In [14]:
from dataclasses import dataclass

import sys
import os

root_dir = os.path.abspath(os.path.join(os.path.dirname('__file__'), '../..'))
sys.path.append(root_dir)


In [15]:
import sqlite3

# Connect to the SQLite3 database in read-only mode
conn = sqlite3.connect('file:E:/.adds/koreader/settings/statistics.sqlite3?mode=ro', uri=True)

# Create a cursor object to interact with the database
cursor = conn.cursor()



In [16]:
cursor.execute('SELECT * FROM book')
rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 'KOReader Quickstart Guide', 'N/A', 0, 1739640383, 0, 8, 'N/A', 'N/A', 'f57f1d89ea1f37a9c43b68371cb7899d', 125, 5)
(2, 'The Shift Omnibus', 'Hugh Howey', 0, 1742509008, 0, 1080, 'N/A', 'en', 'c3a8234c1bf4ee032bec78e42966a263', 19530, 313)
(3, 'Jurassic Park', 'Michael Crichton', 0, 1744756314, 0, 988, 'N/A', 'und', '3769de9df0fe1c978fdf8416ff924b1c', 42229, 974)
(4, 'Sphere', 'Michael Crichton', 0, 1746453853, 0, 665, 'N/A', 'en-US', '76b3392631b316b4491457c0d6400efc', 29070, 573)
(5, 'Learning Statistics with Python \x14 Learning Statistics with Python', 'Graham Pople', 0, 1746389060, 0, 1, 'N/A', 'N/A', '61c13298740e4a0d9932b58b0d4250b5', None, 0)


In [17]:
from datetime import datetime, timedelta
from dataclasses import dataclass

cursor.execute('SELECT * FROM page_stat_data psd LEFT JOIN book b ON b.id == psd.id_book')
rows = cursor.fetchall()

@dataclass
class Row:
    book_id: str
    # title: str
    # author: str
    page: int
    start_time: datetime
    duration: str
    time_difference: timedelta

converted_rows: [Row] = []

for row in rows:
    start_time_epoch = row[2]  # Assuming the start_time is in the second column
    duration_seconds = row[3]  # Assuming the duration is in the third column
    book_title = row[6]
    book_author = row[7]

    # Convert epoch time to human-readable format
    start_time = datetime.fromtimestamp(start_time_epoch)

    # Convert duration from seconds to a timedelta object
    duration = str(timedelta(seconds=duration_seconds))

    # converted_rows.append(Row(row[0], book_title, book_author, row[1], start_time, duration, None))
    converted_rows.append(Row(row[0], row[1], start_time, duration, None))

# Print the converted rows
converted_rows = sorted(converted_rows, key=lambda x: x.start_time)

previous_start_time = None

for row in converted_rows:
    start_time = row.start_time

    if row.duration == '0:02:00':
        continue

    if previous_start_time:
        #current_start_time = datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
        row.time_difference = start_time - previous_start_time
        previous_start_time = start_time
    else:
        row.time_difference = 'N/A'
        previous_start_time = start_time # datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')




In [18]:

for row in converted_rows:
    if row.duration == '0:02:00':
        continue

    print(row.start_time, row.duration, row.time_difference)

# Close the connection

2025-02-15 17:16:48 0:00:09 N/A
2025-02-15 17:17:02 0:00:05 0:00:14
2025-02-15 17:17:08 0:00:16 0:00:06
2025-02-15 17:17:27 0:00:10 0:00:19
2025-02-15 17:24:58 0:00:07 0:07:31
2025-02-15 17:25:05 0:00:53 0:00:07
2025-02-15 17:25:58 0:00:25 0:00:53
2025-02-15 17:29:25 0:00:05 0:03:27
2025-02-15 17:29:32 0:00:37 0:00:07
2025-02-15 17:30:09 0:00:23 0:00:37
2025-02-15 17:30:37 0:00:59 0:00:28
2025-02-15 21:39:10 0:00:12 4:08:33
2025-02-15 21:41:50 0:00:06 0:02:40
2025-02-15 21:42:13 0:00:23 0:00:23
2025-02-15 21:42:37 0:00:14 0:00:24
2025-02-15 21:42:51 0:00:09 0:00:14
2025-02-15 22:21:23 0:00:16 0:38:32
2025-02-15 22:21:40 0:01:25 0:00:17
2025-02-15 22:23:05 0:01:20 0:01:25
2025-02-15 22:24:25 0:00:32 0:01:20
2025-02-15 22:24:57 0:00:32 0:00:32
2025-02-15 22:28:43 0:00:13 0:03:46
2025-02-15 22:40:59 0:01:00 0:12:16
2025-02-15 22:42:19 0:00:11 0:01:20
2025-02-15 22:42:33 0:00:05 0:00:14
2025-02-15 22:42:39 0:00:05 0:00:06
2025-02-15 22:42:44 0:01:09 0:00:05
2025-02-15 22:43:53 0:00:19 0:01

In [19]:
grouped_rows = {}

def get_day_and_night_of(start_time):
    if start_time.hour < 4:
        return (start_time - timedelta(days=1)).strftime('%Y-%m-%d')
    else:
        return start_time.strftime('%Y-%m-%d')

for row in converted_rows:
    day_and_night_of = get_day_and_night_of(row.start_time)
    if day_and_night_of not in grouped_rows:
        grouped_rows[day_and_night_of] = []
    grouped_rows[day_and_night_of].append(row)

len(grouped_rows)

68

In [ ]:
sleep_events.load_eeg_events(print)

In [20]:
grouped_rows['2025-03-05']

[Row(book_id=2, page=738, start_time=datetime.datetime(2025, 3, 5, 22, 7, 49), duration='0:02:00', time_difference=None),
 Row(book_id=2, page=739, start_time=datetime.datetime(2025, 3, 5, 22, 15, 32), duration='0:00:57', time_difference=datetime.timedelta(seconds=83537)),
 Row(book_id=2, page=740, start_time=datetime.datetime(2025, 3, 5, 22, 16, 29), duration='0:02:00', time_difference=None),
 Row(book_id=2, page=741, start_time=datetime.datetime(2025, 3, 5, 22, 21, 1), duration='0:00:46', time_difference=datetime.timedelta(seconds=329)),
 Row(book_id=2, page=742, start_time=datetime.datetime(2025, 3, 5, 22, 21, 47), duration='0:00:38', time_difference=datetime.timedelta(seconds=46)),
 Row(book_id=2, page=743, start_time=datetime.datetime(2025, 3, 5, 22, 22, 25), duration='0:02:00', time_difference=None),
 Row(book_id=2, page=744, start_time=datetime.datetime(2025, 3, 5, 22, 25, 11), duration='0:01:23', time_difference=datetime.timedelta(seconds=204)),
 Row(book_id=2, page=745, start_

In [21]:
import sleep_events
from importlib import reload
reload(sleep_events)
from sleep_events import connect_to_firebase

db = connect_to_firebase()

In [23]:
import json
from dataclasses import asdict
import sys

# Assuming grouped_rows is already populated
for day, rows in grouped_rows.items():
    json_rows = []
    for row in rows:
        # Kobo caps at 2 minutes.  So these might be where I just opened the Kobo and then did something else for a while.  Or maybe I was just not really paying attention
        # and getting through the page quickly.
        # If exclude these not good (maybe - I don't recall exactly) for these nights:
        # http://localhost:3000/data/screenTracking/explorer?date=2025-03-05
        # if row.duration == '0:02:00':
        #     continue

        row_dict = asdict(row)
        row_dict['start_time'] = row.start_time.isoformat()
        row_dict['time_difference'] = str(row.time_difference)
        row_dict['day_and_night_of'] = day
        json_rows.append(row_dict)

    json_blob = json.dumps(json_rows, indent=4)
    #print(f"JSON blob for {day}")
    #print(json_blob)

    # Get the size of json_blob in bytes
    json_blob_size = sys.getsizeof(json_blob)
    print(f"Size of json_blob for {day}: {json_blob_size} bytes")

    db.collection('kobo').document(day).set({'data': json_rows})

Size of json_blob for 2025-02-15: 10964 bytes


RetryError: Timeout of 60.0s exceeded, last exception: 503 Getting metadata from plugin failed with error: ('invalid_grant: Invalid JWT: Token must be a short-lived token (60 minutes) and in a reasonable timeframe. Check your iat and exp values in the JWT claim.', {'error': 'invalid_grant', 'error_description': 'Invalid JWT: Token must be a short-lived token (60 minutes) and in a reasonable timeframe. Check your iat and exp values in the JWT claim.'})